In [1]:
### Plotting bettis

script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using CSV
using JLD
using DataFrames
using JSON
using MAT

include("helper_functions.jl")
println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-605082635389536698\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8344588319191693229\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-605082635389536698\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


importing packages...
packages and functions imported
Elapsed time = 15.46858286857605 seconds 
 


In [2]:
### Set parameters

betti_colors = [["#243a4c"] ["#406372"] ["#66939e"] ["#9bc3c6"]]

# Read from config file
config_file = "config101220.json"
config = read_config("$(homedir())/configs/$(config_file)")

# Parameters for all graphs
const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]


### Locate data and find nametags
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("_bettis",x), readdir(read_dir))
betti_files = filter(x->occursin(DATE_STRING, x), betti_files)
betti_files = filter(x->!occursin("dsi", x), betti_files)

println("Located the following graph files:")
# for betti_file in betti_files
#     println(betti_file)
# end


# Locate the nametags
nametags = []
for betti_file in betti_files
    println(betti_file)
    tag = split(split(betti_file, "$(DATE_STRING)_")[2], "_bettis")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)


for nametag in nametags
    namefiles = filter(x->occursin(nametag,x), betti_files)
    if length(namefiles)<12
    println(nametag)
    println(length(namefiles))
    end
end

nametags

Located the following graph files:
IID_70nodes_500reps_101220_forward_bettis.jld
IID_70nodes_500reps_101220_overlap_a01_b03_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a02_b04_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a03_b05_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a04_b06_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a05_b07_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a06_b08_overlap_bettis.jld
IID_70nodes_500reps_101220_overlap_a07_b09_overlap_bettis.jld
IID_70nodes_500reps_101220_thresh015_edge363_noiseOnly_bettis.jld
IID_70nodes_500reps_101220_thresh015_edge363_randomized_threshold_bettis.jld
IID_70nodes_500reps_101220_thresh015_edge363_threshold_bettis.jld
IID_70nodes_500reps_101220_thresh01_edge242_noiseOnly_bettis.jld
IID_70nodes_500reps_101220_thresh01_edge242_randomized_threshold_bettis.jld
IID_70nodes_500reps_101220_thresh01_edge242_threshold_bettis.jld
IID_70nodes_500reps_101220_thresh025_edge604_noiseOnly_bettis.jld
IID_

clique10_70nodes_500reps_10_101220_thresh035_edge846_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh03_edge725_noiseOnly_bettis.jld
clique10_70nodes_500reps_10_101220_thresh03_edge725_randomized_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh03_edge725_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh045_edge1087_noiseOnly_bettis.jld
clique10_70nodes_500reps_10_101220_thresh045_edge1087_randomized_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh045_edge1087_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh04_edge966_noiseOnly_bettis.jld
clique10_70nodes_500reps_10_101220_thresh04_edge966_randomized_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh04_edge966_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh055_edge1329_noiseOnly_bettis.jld
clique10_70nodes_500reps_10_101220_thresh055_edge1329_randomized_threshold_bettis.jld
clique10_70nodes_500reps_10_101220_thresh055_edge1329_threshold_betti

clique25_70nodes_500reps_25_101220_thresh01_edge242_randomized_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh01_edge242_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh025_edge604_noiseOnly_bettis.jld
clique25_70nodes_500reps_25_101220_thresh025_edge604_randomized_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh025_edge604_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh02_edge483_noiseOnly_bettis.jld
clique25_70nodes_500reps_25_101220_thresh02_edge483_randomized_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh02_edge483_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh035_edge846_noiseOnly_bettis.jld
clique25_70nodes_500reps_25_101220_thresh035_edge846_randomized_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh035_edge846_threshold_bettis.jld
clique25_70nodes_500reps_25_101220_thresh03_edge725_noiseOnly_bettis.jld
clique25_70nodes_500reps_25_101220_thresh03_edge725_randomized_threshold_b

clique30_70nodes_500reps_30_101220_thresh075_edge1812_randomized_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh075_edge1812_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh07_edge1691_noiseOnly_bettis.jld
clique30_70nodes_500reps_30_101220_thresh07_edge1691_randomized_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh07_edge1691_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh085_edge2053_noiseOnly_bettis.jld
clique30_70nodes_500reps_30_101220_thresh085_edge2053_randomized_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh085_edge2053_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh08_edge1932_noiseOnly_bettis.jld
clique30_70nodes_500reps_30_101220_thresh08_edge1932_randomized_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh08_edge1932_threshold_bettis.jld
clique30_70nodes_500reps_30_101220_thresh09_edge2174_noiseOnly_bettis.jld
clique30_70nodes_500reps_30_101220_thresh09_edge2174_randomized

clique_70nodes_500reps_101220_thresh03_edge725_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh045_edge1087_noiseOnly_bettis.jld
clique_70nodes_500reps_101220_thresh045_edge1087_randomized_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh045_edge1087_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh04_edge966_noiseOnly_bettis.jld
clique_70nodes_500reps_101220_thresh04_edge966_randomized_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh04_edge966_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh055_edge1329_noiseOnly_bettis.jld
clique_70nodes_500reps_101220_thresh055_edge1329_randomized_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh055_edge1329_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh05_edge1208_noiseOnly_bettis.jld
clique_70nodes_500reps_101220_thresh05_edge1208_randomized_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh05_edge1208_threshold_bettis.jld
clique_70nodes_500reps_101220_thresh065_edge1570_noiseOnl

...

wprobTriangle4_70nodes_500reps_04_101220_thresh01_edge242_randomized_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh01_edge242_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh025_edge604_noiseOnly_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh025_edge604_randomized_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh025_edge604_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh02_edge483_noiseOnly_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh02_edge483_randomized_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh02_edge483_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh035_edge846_noiseOnly_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh035_edge846_randomized_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh035_edge846_threshold_bettis.jld
wprobTriangle4_70nodes_500reps_04_101220_thresh03_edge725_noiseOnly_bettis.jld
wp

wprobTriangle6_70nodes_500reps_06_101220_thresh04_edge966_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh055_edge1329_noiseOnly_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh055_edge1329_randomized_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh055_edge1329_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh05_edge1208_noiseOnly_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh05_edge1208_randomized_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh05_edge1208_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh065_edge1570_noiseOnly_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh065_edge1570_randomized_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh065_edge1570_threshold_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh06_edge1449_noiseOnly_bettis.jld
wprobTriangle6_70nodes_500reps_06_101220_thresh06_edge1449_randomized_threshold_be

wprobTriangle8_70nodes_500reps_08_101220_thresh085_edge2053_noiseOnly_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh085_edge2053_randomized_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh085_edge2053_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh08_edge1932_noiseOnly_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh08_edge1932_randomized_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh08_edge1932_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh09_edge2174_noiseOnly_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh09_edge2174_randomized_threshold_bettis.jld
wprobTriangle8_70nodes_500reps_08_101220_thresh09_edge2174_threshold_bettis.jld
wprobTriangle95_70nodes_500reps_095_101220_forward_bettis.jld
wprobTriangle95_70nodes_500reps_095_101220_thresh015_edge363_noiseOnly_bettis.jld
wprobTriangle95_70nodes_500reps_095_101220_thresh015_edge363_randomized_threshold_bettis.jld
wprobTr

wprobTriangle_70nodes_500reps_1_101220_thresh02_edge483_noiseOnly_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh02_edge483_randomized_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh02_edge483_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh035_edge846_noiseOnly_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh035_edge846_randomized_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh035_edge846_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh03_edge725_noiseOnly_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh03_edge725_randomized_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh03_edge725_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh045_edge1087_noiseOnly_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh045_edge1087_randomized_threshold_bettis.jld
wprobTriangle_70nodes_500reps_1_101220_thresh045_edge1087_threshold_bettis.jld
wprobTriangle_70nodes_5

59-element Array{Any,1}:
 "forward"                                
 "overlap_a01_b03_overlap"                
 "overlap_a02_b04_overlap"                
 "overlap_a03_b05_overlap"                
 "overlap_a04_b06_overlap"                
 "overlap_a05_b07_overlap"                
 "overlap_a06_b08_overlap"                
 "overlap_a07_b09_overlap"                
 "thresh015_edge363_noiseOnly"            
 "thresh015_edge363_randomized_threshold" 
 "thresh015_edge363_threshold"            
 "thresh01_edge242_noiseOnly"             
 "thresh01_edge242_randomized_threshold"  
 ⋮                                        
 "thresh07_edge1691_noiseOnly"            
 "thresh07_edge1691_randomized_threshold" 
 "thresh07_edge1691_threshold"            
 "thresh085_edge2053_noiseOnly"           
 "thresh085_edge2053_randomized_threshold"
 "thresh085_edge2053_threshold"           
 "thresh08_edge1932_noiseOnly"            
 "thresh08_edge1932_randomized_threshold" 
 "thresh08_edge1932_threshold

In [9]:
collect(1:5:100)

20-element Array{Int64,1}:
  1
  6
 11
 16
 21
 26
 31
 36
 41
 46
 51
 56
 61
 66
 71
 76
 81
 86
 91
 96

In [17]:

####### Writing for JSON
using JSON

## Filter to include JUST the threshold ones
thresh_files = filter(x -> occursin("threshold",x),betti_files)
thresh_files = filter(x -> !occursin("randomized",x),thresh_files)
thresh_files = filter(x -> !occursin("clique",x),thresh_files)
thresh_files = filter(x -> !occursin("Triangle",x),thresh_files)
thresh_files = filter(x -> !occursin("noise",x),thresh_files)

## Filter to only include every kth data point - for faster visualization loading
k = 3


model_names = []
for thresh_file in thresh_files
    name = split(thresh_file, "_")[1]
    model_names = [model_names; name]
end

unique!(model_names)
betti_names = ["IID","assortative","coreperiphery", "cosineGeometric","disassortative", "discreteUniform","dotProduct", "geometricConf", "randomGeometric", "ringLattice", "rmsd", "squaredEuclidean" ];

big_dict = Dict()
df = DataFrame(edge = collect(1:binomial(NNODES,2)))

for (i,model_name) in enumerate(model_names)



    model_dict = Dict()

    # Run through all the threshold files and extract Betti curves

    model_thresh_files = filter(x->startswith(x,model_name), thresh_files)

    

    for model_thresh_file in model_thresh_files
        

        rho_dict = Dict()

        bettisArray = load("../processed_data/results/70nodes/$(model_thresh_file)","bettisArray")
        bettisArrayAvg = dropdims(mean(bettisArray, dims=1), dims=1)
        bettisArrayStd = dropdims(std(bettisArray,dims=1), dims=1)
        
        # Keep only ever kth datapoint
        nEdges = size(bettisArray)[2]
        keep_data = collect(1:k:nEdges)

        
        # Extract the rho value or edge number and add to dictionary
        thresh_edge = parse(Int,split(split(model_thresh_file,"edge")[2], "_")[1])
        rho = replace(split(split(model_thresh_file,"_edge")[1], "_thresh")[2], "."=> "")
        println(thresh_edge)


        for dim in 1:MAXDIM
            rho_dict["dim$(dim)"] = bettisArrayAvg[keep_data,dim]
#             rho_dict["std$(dim)"] = bettisArrayStd[:, dim]
            df[!, "$(betti_names[i])_thresh$(rho)_dim$(dim)"] = bettisArrayAvg[:, dim]
        end




        model_dict["$(thresh_edge)"] = rho_dict

    end


    big_dict["$(betti_names[i])"] = model_dict
    
    println("finished $(model_name)")
    
end

big_dict





# df

363
242
604
483
846
725
1087
966
1329
1208
1570
1449
1812
1691
2053
1932
2174
finished IID
363
242
604
483
846
725
1087
966
1329
1208
1570
1449
1812
1691
2053
1932
2174
finished assortative
363
242
604
483
846
725
1087
966
1329
1208
1570
1449
1812
1691
2053
1932
2174
finished coreperiphery
363
242
604
483
846
725
1087
966
1329
1208
1570
1449
1812
1691
2053
1932
2174
finished cosineGeometric
363
242
604
483
846
725
1087
966
1329
1208
1570
1449
1812
1691
2053
1932
2174
finished disassortative
363
242
604
483
846
725
1087
966
1329
1208
1570
1449
1812
1691
2053
1932
2174
finished discreteUniformConf


InterruptException: InterruptException:

In [8]:
big_dict

Dict{Any,Any} with 12 entries:
  "discreteUniform"  => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "cosineGeometric"  => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "dotProduct"       => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "geometricConf"    => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "disassortative"   => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "assortative"      => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "rmsd"             => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "squaredEuclidean" => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "ringLattice"      => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "IID"              => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "randomGeometric"  => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 0…
  "coreperiphery"    => Dict{Any,Any}("966"=>Dict{Any,Any}("dim2"=>[0.0, 0.0, 

In [ ]:
CSV.write("test_bettis.csv",df)

In [16]:
## Write to json
open("../templates/main_k3.json", "w") do f
    JSON.print(f, big_dict)
end
println("done saving")

done saving


## Loading and wrapping classification mat files


In [22]:
model_names = ["IID","assortative","coreperiphery", "cosineGeometric","disassortative", "discreteUniform",
    "dotProduct", "geometricConf", "randomGeometric", "ringLattice", "rmsd", "squaredEuclidean" ]
mat_dir = "../processed_data/classification/"
mat_files = readdir(mat_dir)
mat_files = filter(x -> occursin(".mat",x), mat_files)

17-element Array{String,1}:
 "matrix_all_0.1.mat" 
 "matrix_all_0.15.mat"
 "matrix_all_0.2.mat" 
 "matrix_all_0.25.mat"
 "matrix_all_0.3.mat" 
 "matrix_all_0.35.mat"
 "matrix_all_0.4.mat" 
 "matrix_all_0.45.mat"
 "matrix_all_0.5.mat" 
 "matrix_all_0.55.mat"
 "matrix_all_0.6.mat" 
 "matrix_all_0.65.mat"
 "matrix_all_0.7.mat" 
 "matrix_all_0.75.mat"
 "matrix_all_0.8.mat" 
 "matrix_all_0.85.mat"
 "matrix_all_0.9.mat" 

In [23]:
# Make a n_threhsolds x n_models^2 matrix

mat1 = matread("$(mat_dir)/$(mat_files[1])")
mat1["C"]

12×12 Array{Float64,2}:
 161.0    0.0    1.0    0.0   88.0    0.0  …    0.0   0.0    0.0   0.0   0.0
   0.0  250.0    0.0    0.0    0.0    0.0       0.0   0.0    0.0   0.0   0.0
   6.0    0.0  243.0    0.0    1.0    0.0       0.0   0.0    0.0   0.0   0.0
   0.0    0.0    0.0  149.0    0.0    0.0       0.0  28.0   39.0  17.0  17.0
 109.0    0.0    0.0    0.0  141.0    0.0       0.0   0.0    0.0   0.0   0.0
   0.0    0.0    0.0    1.0    0.0  211.0  …    6.0   1.0    0.0   0.0   0.0
   0.0    0.0    0.0    0.0    0.0   15.0     103.0   0.0    0.0   0.0   0.0
   0.0    0.0    0.0    0.0    0.0    0.0     151.0   0.0    0.0   0.0   0.0
   0.0    0.0    0.0   42.0    0.0    0.0       0.0  58.0   24.0  69.0  57.0
   0.0    0.0    0.0   27.0    0.0    0.0       0.0  13.0  180.0  23.0   7.0
   0.0    0.0    0.0   30.0    0.0    0.0  …    0.0  62.0   32.0  60.0  66.0
   0.0    0.0    0.0   36.0    0.0    0.0       0.0  65.0   39.0  65.0  45.0

In [ ]:
col_names = ["$(a)_$(b)" for a in model_names for b in model_names]

In [25]:

df = DataFrame(mat1["C"])
rename!(df,Symbol.(model_names))
df.true_class = model_names
df

,IID,assortative,coreperiphery,cosineGeometric,disassortative,discreteUniform
,Float64,Float64,Float64,Float64,Float64,Float64
1,161.0,0.0,1.0,0.0,88.0,0.0
2,0.0,250.0,0.0,0.0,0.0,0.0
3,6.0,0.0,243.0,0.0,1.0,0.0
4,0.0,0.0,0.0,149.0,0.0,0.0
5,109.0,0.0,0.0,0.0,141.0,0.0
6,0.0,0.0,0.0,1.0,0.0,211.0
7,0.0,0.0,0.0,0.0,0.0,15.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,42.0,0.0,0.0


In [26]:
df2 = stack(df,1:12)
rename!(df2, [:trueClass, :predictedClass, :accuracy])

,trueClass,predictedClass,accuracy
,String,Cat…,Float64
1,IID,IID,161.0
2,assortative,IID,0.0
3,coreperiphery,IID,6.0
4,cosineGeometric,IID,0.0
5,disassortative,IID,109.0
6,discreteUniform,IID,0.0
7,dotProduct,IID,0.0
8,geometricConf,IID,0.0
9,randomGeometric,IID,0.0


In [27]:
df2.truePredicted = ["$(b)_$(a)" for a in model_names for b in model_names]
df2

,trueClass,predictedClass,accuracy,truePredicted
,String,Cat…,Float64,String
1,IID,IID,161.0,IID_IID
2,assortative,IID,0.0,assortative_IID
3,coreperiphery,IID,6.0,coreperiphery_IID
4,cosineGeometric,IID,0.0,cosineGeometric_IID
5,disassortative,IID,109.0,disassortative_IID
6,discreteUniform,IID,0.0,discreteUniform_IID
7,dotProduct,IID,0.0,dotProduct_IID
8,geometricConf,IID,0.0,geometricConf_IID
9,randomGeometric,IID,0.0,randomGeometric_IID


In [28]:
# Get the two vectors we care about
df3 = DataFrame(TP = df2.truePredicted)


,TP
,String
1,IID_IID
2,assortative_IID
3,coreperiphery_IID
4,cosineGeometric_IID
5,disassortative_IID
6,discreteUniform_IID
7,dotProduct_IID
8,geometricConf_IID
9,randomGeometric_IID


In [30]:
# repeat

for mat_file in mat_files
    mat = matread("$(mat_dir)/$(mat_file)")["C"]
    thresh = "thresh$(replace(split(split(mat_file, ".mat")[1], "_")[3], "."=>""))"
    df = stack(DataFrame(mat),1:12)
    println(thresh)
    df3[!, thresh] = df.value
end
df3

thresh01
thresh015
thresh02
thresh025
thresh03
thresh035
thresh04
thresh045
thresh05
thresh055
thresh06
thresh065
thresh07
thresh075
thresh08
thresh085
thresh09


,TP,thresh01,thresh015,thresh02,thresh025,thresh03
,String,Float64,Float64,Float64,Float64,Float64
1,IID_IID,161.0,178.0,204.0,231.0,250.0
2,assortative_IID,0.0,0.0,0.0,0.0,0.0
3,coreperiphery_IID,6.0,1.0,0.0,0.0,0.0
4,cosineGeometric_IID,0.0,0.0,0.0,0.0,0.0
5,disassortative_IID,109.0,72.0,41.0,16.0,0.0
6,discreteUniform_IID,0.0,0.0,0.0,0.0,0.0
7,dotProduct_IID,0.0,0.0,0.0,0.0,0.0
8,geometricConf_IID,0.0,0.0,0.0,0.0,0.0
9,randomGeometric_IID,0.0,0.0,0.0,0.0,0.0


In [31]:
CSV.write("classification_all_101220.csv",df3)

"classification_all_101220.csv"